In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import math
import json
import numpy as np
import tensorflow as tf
import random
from sklearn.neighbors import KNeighborsClassifier
import timeit

In [3]:
path = "/home/sukun/ucla_github/"

# Preprocess data

## load data

In [4]:
def load_data(path):
    with open(path) as f:
        data = json.load(f)
    return data

# load ucla data

In [5]:
def load_ucla_data(path):
    train_1 = load_data(path+'new_xyz_transform_skel_1.json')
    train_2 = load_data(path+'new_xyz_transform_skel_2.json')
    train_data = {**train_1,**train_2}
    # train data
    for key in train_data.keys():
        train_data[key] = np.asarray(train_data[key])
    
    # test data
    test_data = load_data(path+'new_xyz_transform_skel_3.json')
    for key in test_data.keys():
        test_data[key] = np.asarray(test_data[key])
    # delete bad data
    del train_data['a02_s09_e04_v02']
    del test_data['a02_s09_e04_v03']
    # size of training and test data
    print("Size of training data: ", len(train_data))
    print("Size of test data: ", len(test_data))
    return train_data, test_data

## Normalize ucla data

In [6]:
def normalize_ucla(video):
    max_60 = np.amax(video, axis=0)
    min_60 = np.amin(video, axis=0)
    max_x = np.max([max_60[i] for i in range(0,60,3)])
    max_y = np.max([max_60[i] for i in range(1,60,3)])
    max_z = np.max([max_60[i] for i in range(2,60,3)])
    min_x = np.min([min_60[i] for i in range(0,60,3)])
    min_y = np.min([min_60[i] for i in range(1,60,3)])
    min_z = np.min([min_60[i] for i in range(2,60,3)])
    norm = np.zeros_like(video)
    for i in range(0,60,3):
        norm[:,i] = 2*(video[:,i]-min_x)/(max_x-min_x)-1
        norm[:,i+1] = 2*(video[:,i+1]-min_y)/(max_y-min_y)-1
        norm[:,i+2] = 2*(video[:,i+2]-min_z)/(max_z-min_z)-1
    return norm


## downsample ucla data

In [7]:
def downsample_ucla(data):
    dsamp = dict()
    for key, val in data.items():
        if val.shape[0] > 50:
            new_val = np.zeros((50, 60))
            diff = math.floor(val.shape[0]/50)
            idx = 0
            for i in range(0, val.shape[0], diff):
                new_val[idx, :] = val[i, :]
                idx += 1
                if idx >= 50:
                    break
            dsamp.update({key: new_val})
        else:
            dsamp.update({key: val})
    return dsamp

## get feature, label pairs in dataset

In [8]:
def get_feature_label(raw_data,dsamp_data,max_len=50):
    fea_xyz = []
    labels = []
    seq_len = []
    for key, val in raw_data.items():
        label = int(key[1:3])
        if label ==11:
            label = 7
        elif label == 12:
            label = 10
        label -= 1
        raw_len = val.shape[0]
        if raw_len > max_len:
            seq_len.append(max_len)
            fea_xyz.append(dsamp_data[key])
        else:
            seq_len.append(raw_len)
            pad_data = np.zeros((max_len,60))
            pad_data[:raw_len,:] = dsamp_data[key]
            fea_xyz.append(pad_data)
        one_hot_label = np.zeros((10,))
        one_hot_label[label] = 1.
        labels.append(one_hot_label)
    return fea_xyz,labels,seq_len

## Complete preprocess for ucla dataset

In [9]:
def preprocess_ucla(path):
    # raw data
    train_data, test_data = load_ucla_data(path)
    # normalize
    for key in train_data.keys():
        train_data[key] = normalize_ucla(train_data[key])
    for key in test_data.keys():
        test_data[key] = normalize_ucla(test_data[key])
    # down sample
    dsamp_train = downsample_ucla(train_data)
    dsamp_test = downsample_ucla(test_data)
    # get features and labels pair
    tr_fea_xyz,tr_label,tr_seq_len = get_feature_label(train_data,dsamp_train,max_len=50)
    train_label = [np.argmax(tr_label[i]) for i in range(len(tr_label))]
    te_fea_xyz,te_label,te_seq_len = get_feature_label(test_data,dsamp_test,max_len=50)
    test_label = [np.argmax(te_label[i]) for i in range(len(te_label))]
    return dsamp_train, dsamp_test, tr_fea_xyz, tr_label,tr_seq_len, te_fea_xyz, te_label, te_seq_len

## One line code

In [10]:
dsamp_train, dsamp_test, tr_fea_xyz, tr_label,tr_seq_len, te_fea_xyz, te_label, te_seq_len = preprocess_ucla(path)

Size of training data:  1019
Size of test data:  463


# Seq2Seq class

# FS class

In [11]:
from tensorflow.python.ops.rnn import _transpose_batch_time
class Seq2SeqModelFS(object):
    def __init__(self, max_seq_len, input_size, rnn_size, batch_size, lr, train_keep_prob,decay_rate=0.95,dtype=tf.float32):
        self.max_seq_len = max_seq_len
        self.rnn_size = rnn_size
        self.batch_size = tf.placeholder_with_default(batch_size,shape=())
        self.input_size = input_size
        self.lr = tf.Variable( float(lr), trainable=False, dtype=dtype )
        self.learning_rate_decay_op = self.lr.assign( self.lr * decay_rate)
        self.keep_prob = tf.placeholder_with_default(1.0,shape=())
        self.global_step = tf.Variable(0, trainable=False)
        print('rnn_size = {0}'.format(rnn_size))
        
        with tf.variable_scope("inputs"):
            self.enc_xyz = tf.placeholder(dtype, shape=[None, self.max_seq_len, input_size], name='enc_xyz')
            self.dec_xyz = tf.placeholder(dtype, shape=[None, self.max_seq_len, input_size], name='dec_xyz')
            self.seq_len = tf.placeholder(tf.int32,[None])
            mask = tf.sign(tf.reduce_max(tf.abs(self.enc_xyz[:,1:,:]), 2))

        with tf.variable_scope("prediction"):
            with tf.variable_scope("encoder"):
                with tf.variable_scope("encoder_xyz",reuse=tf.AUTO_REUSE):
                    cell_fw_xyz = [tf.nn.rnn_cell.GRUCell(self.rnn_size//2) for i in range(3)]
                    cell_bw_xyz = [tf.nn.rnn_cell.GRUCell(self.rnn_size//2) for i in range(3)]
                    tuple_xyz = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cell_fw_xyz,cell_bw_xyz,self.enc_xyz,dtype=tf.float32,sequence_length=self.seq_len)
                    bi_xyz_h = tf.concat((tuple_xyz[1][-1],tuple_xyz[2][-1]),-1)
                    self.bi_xyz_h = bi_xyz_h
            self.knn_state = self.bi_xyz_h
            with tf.variable_scope("decoder"):
                with tf.variable_scope("decoder_xyz",reuse=tf.AUTO_REUSE):
                    cell_xyz__ = tf.nn.rnn_cell.GRUCell(self.rnn_size)
                    cell_xyz_ = LinearSpaceDecoderWrapper(cell_xyz__,self.input_size)
                    cell_xyz = ResidualWrapper(cell_xyz_)
                    def loop_fn(time, cell_output, cell_state, loop_state):
                        """
                        Loop function that allows to control input to the rnn cell and manipulate cell outputs.
                        :param time: current time step
                        :param cell_output: output from previous time step or None if time == 0
                        :param cell_state: cell state from previous time step
                        :param loop_state: custom loop state to share information between different iterations of this loop fn
                        :return: tuple consisting of
                          elements_finished: tensor of size [bach_size] which is True for sequences that have reached their end,
                            needed because of variable sequence size
                          next_input: input to next time step
                          next_cell_state: cell state forwarded to next time step
                          emit_output: The first return argument of raw_rnn. This is not necessarily the output of the RNN cell,
                            but could e.g. be the output of a dense layer attached to the rnn layer.
                          next_loop_state: loop state forwarded to the next time step
                        """
                        if cell_output is None:
                            # time == 0, used for initialization before first call to cell
                            next_cell_state = self.bi_xyz_h
                            # the emit_output in this case tells TF how future emits look
                            emit_output = tf.zeros([self.input_size])
                        else:
                            # t > 0, called right after call to cell, i.e. cell_output is the output from time t-1.
                            # here you can do whatever ou want with cell_output before assigning it to emit_output.
                            # In this case, we don't do anything
                            next_cell_state = self.bi_xyz_h#NOTE:IF NO-FS, use cell_state#
                            emit_output = cell_output  

                        # check which elements are finished
                        elements_finished = (time >= self.seq_len-1)
                        finished = tf.reduce_all(elements_finished)

                        # assemble cell input for upcoming time step
                        current_output = emit_output if cell_output is not None else None
                        #input_original = inputs_ta.read(time)  # tensor of shape (None, input_dim)
                        input_original = self.enc_xyz[:,0,:]
                        if current_output is None:
                            # this is the initial step, i.e. there is no output from a previous time step, what we feed here
                            # can highly depend on the data. In this case we just assign the actual input in the first time step.
                            next_in = input_original
                        else:
                            # time > 0, so just use previous output as next input
                            # here you could do fancier things, whatever you want to do before passing the data into the rnn cell
                            # if here you were to pass input_original than you would get the normal behaviour of dynamic_rnn
                            next_in = current_output

                        next_input = tf.cond(finished,
                                             lambda: tf.zeros([self.batch_size, self.input_size], dtype=tf.float32),  # copy through zeros
                                             lambda: next_in)  # if not finished, feed the previous output as next input

                        # set shape manually, otherwise it is not defined for the last dimensions
                        next_input.set_shape([None, self.input_size])

                        # loop state not used in this example
                        next_loop_state = None
                        return (elements_finished, next_input, next_cell_state, emit_output, next_loop_state)
                    outputs_ta, def_final_state_xyz, _ = tf.nn.raw_rnn(cell_xyz, loop_fn)
                    self.dec_outputs_xyz = _transpose_batch_time(outputs_ta.stack())
            def loss_with_mask(pred,gt,mask):
                loss = tf.reduce_sum(tf.abs(pred-gt),2)*mask
                loss = tf.reduce_sum(loss,1)
                loss /= tf.reduce_sum(mask,1)
                loss = tf.reduce_mean(loss)
                return loss
        with tf.variable_scope("pred_xyz",reuse=tf.AUTO_REUSE):
            pred_xyz2xyz = self.dec_outputs_xyz
            self.loss = loss_with_mask(pred_xyz2xyz,self.enc_xyz[:,1:,:],mask)
        
        opt = tf.train.AdamOptimizer(self.lr)
        gradients, self.pred_vars = zip(*opt.compute_gradients(self.loss))
        clipped_gradients, norm = tf.clip_by_global_norm(gradients, 25)
        self.updates = opt.apply_gradients(zip(clipped_gradients,self.pred_vars),global_step = self.global_step)
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)
    
    def step(self, session, encoder_inputs_xyz,decoder_inputs_xyz,seq_len,forward_only):
        if not forward_only:
            input_feed = {self.enc_xyz: encoder_inputs_xyz,
                          self.dec_xyz: decoder_inputs_xyz,
                          self.seq_len: seq_len}
            output_feed = [self.updates,self.loss]
            outputs = session.run(output_feed, input_feed)
            return outputs[0], outputs[1]

# FW class

In [12]:
class Seq2SeqModelFW(object):
    def __init__(self, max_seq_len, input_size, rnn_size, batch_size, lr, train_keep_prob,decay_rate=0.95,dtype=tf.float32):
        self.max_seq_len = max_seq_len
        self.rnn_size = rnn_size
        self.batch_size = tf.placeholder_with_default(batch_size,shape=())
        self.input_size = input_size
        self.lr = tf.Variable( float(lr), trainable=False, dtype=dtype)
        self.learning_rate_decay_op = self.lr.assign( self.lr * decay_rate)
        self.keep_prob = tf.placeholder_with_default(1.0,shape=())

        self.global_step = tf.Variable(0, trainable=False)
        print('rnn_size = {0}'.format(rnn_size))
        
        with tf.variable_scope("inputs"):
            self.enc_xyz = tf.placeholder(dtype, shape=[None, self.max_seq_len, input_size], name='enc_xyz')
            self.dec_xyz = tf.placeholder(dtype, shape=[None, self.max_seq_len, input_size], name='dec_xyz')
            self.seq_len = tf.placeholder(tf.int32,[None])
            mask = tf.sign(tf.reduce_max(tf.abs(self.enc_xyz), 2))
        with tf.variable_scope("prediction"):
            with tf.variable_scope("encoder"):
                with tf.variable_scope("encoder_xyz",reuse=tf.AUTO_REUSE):
                    cell_fw_xyz = [tf.nn.rnn_cell.GRUCell(self.rnn_size//2) for i in range(3)]
                    cell_bw_xyz = [tf.nn.rnn_cell.GRUCell(self.rnn_size//2) for i in range(3)]
                    tuple_xyz = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cell_fw_xyz,cell_bw_xyz,self.enc_xyz,dtype=tf.float32,sequence_length=self.seq_len)
                    bi_xyz_h = tf.concat((tuple_xyz[1][-1],tuple_xyz[2][-1]),-1)
                    self.enc_states = tuple_xyz[0] #all encoder states [batch,time,2048]
                    self.bi_xyz_h = bi_xyz_h
            self.knn_state = self.bi_xyz_h
            with tf.variable_scope("decoder"):
                with tf.variable_scope("decoder_xyz",reuse=tf.AUTO_REUSE):
                    cell_xyz = tf.nn.rnn_cell.GRUCell(self.rnn_size)
                    self.dec_outputs_xyz, dec_final_state_xyz = tf.nn.dynamic_rnn(cell_xyz, tf.zeros_like(self.dec_xyz), sequence_length=self.seq_len,\
                                                                     initial_state = self.bi_xyz_h,dtype = tf.float32)
            def loss_with_mask(pred,gt,mask):
                loss = tf.reduce_sum(tf.abs(pred-gt),2)*mask
                loss = tf.reduce_sum(loss,1)
                loss /= tf.reduce_sum(mask,1)
                loss = tf.reduce_mean(loss)
                return loss
            with tf.variable_scope("pred_xyz",reuse=tf.AUTO_REUSE):
                FC = tf.layers.Dense(units=60,activation=None, name="pred_skel")
                pred_xyz2xyz = FC(self.dec_outputs_xyz)
            self.loss = loss_with_mask(pred_xyz2xyz,self.enc_xyz,mask)
        
        self.pred_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "prediction/encoder")
#         self.fc_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "prediction/pred_xyz")
        params = self.pred_vars #+ self.fc_vars
        opt = tf.train.AdamOptimizer(self.lr)
        gradients, self.pred_vars = zip(*opt.compute_gradients(self.loss))
        clipped_gradients, norm = tf.clip_by_global_norm(gradients, 25)
        self.updates = opt.apply_gradients(zip(clipped_gradients,self.pred_vars),global_step = self.global_step)
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)
    
    def step(self, session, encoder_inputs_xyz,decoder_inputs_xyz,seq_len,forward_only):
        
        if not forward_only:
            input_feed = {self.enc_xyz: encoder_inputs_xyz,
                          self.dec_xyz: decoder_inputs_xyz,
                          self.seq_len: seq_len}
            output_feed = [self.updates,self.loss]
            outputs = session.run(output_feed, input_feed)
            return outputs[0], outputs[1]

## Linear wrapper for FS

In [13]:
from tensorflow.contrib.rnn.python.ops.core_rnn_cell import RNNCell
class LinearSpaceDecoderWrapper(RNNCell):
  """Operator adding a linear encoder to an RNN cell"""

  def __init__(self, cell, output_size):
    """Create a cell with with a linear encoder in space.

    Args:
      cell: an RNNCell. The input is passed through a linear layer.

    Raises:
      TypeError: if cell is not an RNNCell.
    """
    if not isinstance(cell, RNNCell):
      raise TypeError("The parameter cell is not a RNNCell.")

    self._cell = cell

    print( 'output_size = {0}'.format(output_size) )
    print( ' state_size = {0}'.format(self._cell.state_size) )

    # Tuple if multi-rnn
    if isinstance(self._cell.state_size,tuple):

      # Fine if GRU...
      insize = self._cell.state_size[-1]

      # LSTMStateTuple if LSTM
      if isinstance( insize, LSTMStateTuple ):
        insize = insize.h

    else:
      # Fine if not multi-rnn
      insize = self._cell.state_size

    self.w_out = tf.get_variable("proj_w_out",
        [insize, output_size],
        dtype=tf.float32,
        initializer=tf.random_uniform_initializer(minval=-0.04, maxval=0.04))
    self.b_out = tf.get_variable("proj_b_out", [output_size],
        dtype=tf.float32,
        initializer=tf.random_uniform_initializer(minval=-0.04, maxval=0.04))

    self.linear_output_size = output_size


  @property
  def state_size(self):
    return self._cell.state_size

  @property
  def output_size(self):
    return self.linear_output_size

  def __call__(self, inputs, state, scope=None):
    """Use a linear layer and pass the output to the cell."""

    # Run the rnn as usual
    output, new_state = self._cell(inputs, state, scope)

    # Apply the multiplication to everything
    output = tf.matmul(output, self.w_out) + self.b_out

    return output, new_state

## Residual Wrapper for FS

In [14]:
class ResidualWrapper(RNNCell):
  """Operator adding residual connections to a given cell."""

  def __init__(self, cell):
    """Create a cell with added residual connection.

    Args:
      cell: an RNNCell. The input is added to the output.

    Raises:
      TypeError: if cell is not an RNNCell.
    """
    if not isinstance(cell, RNNCell):
      raise TypeError("The parameter cell is not a RNNCell.")

    self._cell = cell

  @property
  def state_size(self):
    return self._cell.state_size

  @property
  def output_size(self):
    return self._cell.output_size

  def __call__(self, inputs, state, scope=None):
    """Run the cell and add a residual connection."""

    # Run the rnn as usual
    output, new_state = self._cell(inputs, state, scope)

    # Add the residual connection
    output = tf.add(output, inputs)

    return output, new_state

# Get learned Features for knn evaluation

In [15]:
def get_feature(model, session, encoder_inputs_xyz, batch_size, seq_len):
    input_feed = {model.enc_xyz: encoder_inputs_xyz,
                  model.dec_xyz: encoder_inputs_xyz,
                  model.seq_len: seq_len, model.batch_size:batch_size}
    output_feed = [model.knn_state]
    outputs = session.run(output_feed, input_feed)
    return outputs[0]

# Training

In [16]:
def get_session():
    """Create a session that dynamically allocates memory."""
    # See: https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
    config = tf.ConfigProto(log_device_placement=True,allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

## Mini batch

In [17]:
def mini_batch(data, seq_len, input_size, batch_size):
    #encoder inputs
    encoder_inputs_xyz = np.zeros((batch_size, seq_len, input_size),dtype=float)
    #length
    seq_len_enc = np.zeros((batch_size,), dtype=float)
    #decoder inputs
    decoder_inputs_xyz = np.zeros((batch_size, seq_len, input_size),dtype=float)
    data_len = len(data)
    
    for i in range(batch_size):
        key = random.choice(list(data.keys()))
        data_sel = data[key]
        encoder_inputs_xyz[i, :data_sel.shape[0], :] = np.copy(data_sel)
        seq_len_enc[i] = data_sel.shape[0]
        
    return encoder_inputs_xyz, decoder_inputs_xyz, seq_len_enc

## Mini batch for evaluating learned features

In [18]:
def mini_batch_classify(feature_xyz,labels,seq_len,batch_size):
    for start in range(0,len(feature_xyz),batch_size):
        end = min(start+batch_size,len(feature_xyz))
        yield feature_xyz[start:end],labels[start:end],seq_len[start:end]

## Hyperparameters

In [19]:
max_seq_len=50
rnn_size=2048
input_size = 60
batch_size=64
lr = .0001
train_keep_prob = 1.0
iterations = 10000

## Training FW

In [20]:
tf.reset_default_graph()
# FW
model = Seq2SeqModelFW(max_seq_len, input_size,rnn_size, batch_size, lr,train_keep_prob)
sess = get_session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Colocations handled automatically by placer.
rnn_size = 2048
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


## FW Training loop 

In [22]:
score = []
loss = []
max_score = 0
start_time = timeit.default_timer()
for i in range(0,iterations+1):
    encoder_inputs_xyz,decoder_inputs_xyz,seq_len_enc = mini_batch(dsamp_train, seq_len=50, input_size=60, batch_size=64)
    _,train_loss = model.step(sess,encoder_inputs_xyz,decoder_inputs_xyz,seq_len_enc, False)
    if i%100 == 0:
        loss.append(train_loss)
        print("step {0}:  train loss:{1:.4f}".format(i, train_loss),end='')
        end_time = timeit.default_timer()
        print(" using {:.2f} sec".format(end_time-start_time))
        start_time = end_time
        knn_feature = []
        j = 0
        for feature_xyz,labels,seq_len in mini_batch_classify(tr_fea_xyz,tr_label,tr_seq_len,batch_size):
            j +=1
            result = get_feature(model, sess, feature_xyz, len(feature_xyz), seq_len)
            knn_feature.append(result)
        knn_feature = np.vstack(knn_feature)
        test_knn_feature = []
        j = 0
        for feature_xyz,labels,seq_len in mini_batch_classify(te_fea_xyz,te_label,te_seq_len,batch_size):
            j +=1
            result = get_feature(model, sess, feature_xyz,len(feature_xyz),seq_len)
            test_knn_feature.append(result)
        test_knn_feature = np.vstack(test_knn_feature)
        neigh = KNeighborsClassifier(n_neighbors=1,metric='cosine')
        neigh.fit(knn_feature, np.argmax(tr_label,axis=1))
        current_score = neigh.score(test_knn_feature, np.argmax(te_label,axis=1))
        print("KNN test score:",current_score)
        score.append(current_score)
        if current_score>max_score:
            max_score = current_score
            model.saver.save(sess,"ucla_model/FW",global_step=i)
    if i%1000 == 0:
        sess.run(model.learning_rate_decay_op)

step 0:  train loss:23.8788 using 2.81 sec
KNN test score: 0.7451403887688985
step 100:  train loss:7.2764 using 74.27 sec
KNN test score: 0.7537796976241901
Instructions for updating:
Use standard file APIs to delete files with this prefix.
step 200:  train loss:6.0994 using 76.06 sec
KNN test score: 0.7580993520518359
step 300:  train loss:5.8068 using 79.12 sec
KNN test score: 0.7602591792656588
step 400:  train loss:5.0657 using 78.89 sec
KNN test score: 0.7645788336933045
step 500:  train loss:5.1571 using 77.96 sec
KNN test score: 0.7861771058315334
step 600:  train loss:4.7311 using 76.71 sec
KNN test score: 0.7991360691144709
step 700:  train loss:4.5220 using 77.87 sec
KNN test score: 0.7926565874730022
step 800:  train loss:4.3473 using 79.54 sec
KNN test score: 0.8120950323974082
step 900:  train loss:4.0523 using 78.28 sec
KNN test score: 0.8056155507559395
step 1000:  train loss:4.0832 using 76.42 sec
KNN test score: 0.8077753779697624
step 1100:  train loss:3.6434 using 7

KeyboardInterrupt: 

# Training FW Autoencoder

### Get trained FW model

In [45]:
sess.close()
tf.reset_default_graph()
model = Seq2SeqModelFW(max_seq_len, 60,rnn_size, batch_size, lr,train_keep_prob)
sess = get_session()
model.saver.restore(sess,"ucla_model/FW-2600") #"FW-$number of step"

rnn_size = 2048
INFO:tensorflow:Restoring parameters from ucla_model/FW-2600


### Retrieve features

In [46]:
knn_feature = []
j = 0
for feature_xyz,labels,seq_len in mini_batch_classify(tr_fea_xyz,tr_label,tr_seq_len,batch_size):
    j +=1
    result = get_feature(model, sess, feature_xyz, len(feature_xyz), seq_len)
    knn_feature.append(result)
knn_feature = np.vstack(knn_feature)

In [47]:
test_knn_feature = []
j = 0
for feature_xyz,labels,seq_len in mini_batch_classify(te_fea_xyz,te_label,te_seq_len,batch_size):
    j +=1
    result = get_feature(model, sess, feature_xyz,len(feature_xyz),seq_len)
    test_knn_feature.append(result)
test_knn_feature = np.vstack(test_knn_feature)

In [49]:
neigh = KNeighborsClassifier(n_neighbors=1,metric='cosine')
neigh.fit(knn_feature,np.argmax(tr_label,axis=1))
t_score = neigh.score(test_knn_feature,np.argmax(te_label,axis=1))
print("after {0} epoch knn score {1:.3f}".format(epoch,t_score))

after 7928 epoch knn score 0.829


## Simple structure for AEC

In [88]:
class AEC(object):
    def __init__(self, input_size, batch_size, lr, dtype=tf.float32):
       
        self.X = tf.placeholder(dtype=tf.float32, shape=[None, input_size], name='X')

        #encoder
        self.fc1 = tf.layers.dense(self.X,1024,activation='tanh')
        self.fc2 = tf.layers.dense(self.fc1,512,activation='tanh')
        self.fea = tf.layers.dense(self.fc2,256,activation='tanh')

        #decoder
        self.fc3 = tf.layers.dense(self.fea,512,activation='tanh')
        self.fc4 = tf.layers.dense(self.fc3,1024,activation='tanh')
        self.Y = tf.layers.dense(self.fc4,input_size,activation=None)

        self.loss = tf.reduce_mean(tf.square(self.X-self.Y))
        self.optimizer = tf.train.AdamOptimizer(lr)
        self.opt = self.optimizer.minimize(self.loss)
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)
    
    def step(self, session, feature, forward_only=False):
        
        if forward_only:
            output = sess.run([self.fea],feed_dict={self.X:feature})
            return output
        else:
            _,train_loss = sess.run([self.opt,self.loss],feed_dict={self.X:feature})
            return train_loss

## AEC training loop

In [89]:
lr = 0.0001
input_size = 2048
batch_size = 16
epochs = 10000

In [90]:
sess.close()
tf.reset_default_graph()
aec_model = AEC(input_size, batch_size, lr)
sess = get_session()
sess.run(tf.global_variables_initializer())

In [91]:
def aec_mini_batch_classify(feature_xyz,batch_size):
    for start in range(0,len(feature_xyz),batch_size):
        end = min(start+batch_size,len(feature_xyz))
        yield feature_xyz[start:end]

In [93]:
aec_score = 0
for epoch in range(epochs):
    for feature in aec_mini_batch_classify(knn_feature,batch_size):
        train_loss = aec_model.step(sess,feature)
#         print("epoch:{0},loss:{1}".format(epoch,train_loss))
    if epoch % 1000 == 0:
        denoise_all = []
        for feature in aec_mini_batch_classify(knn_feature,batch_size):
            denoise = aec_model.step(sess,feature,forward_only=True)
            denoise_all.append(denoise[0])
        see = np.vstack(denoise_all)
            
        test_denoise_all = []
        for feature in aec_mini_batch_classify(test_knn_feature,batch_size):
            denoise = aec_model.step(sess,feature,forward_only=True)
            test_denoise_all.append(denoise[0])
        test_see = np.vstack(test_denoise_all)
        
        neigh = KNeighborsClassifier(n_neighbors=1,metric='cosine')
        neigh.fit(see,np.argmax(tr_label,axis=1))
        t_score = neigh.score(test_see,np.argmax(te_label,axis=1))
        print("after {0} epoch knn score {1:.3f}".format(epoch,t_score))
        if t_score>aec_score:
            aec_score = current_score
            model.saver.save(sess,"ucla_model/FW-AEC")

after 0 epoch knn score 0.842


KeyboardInterrupt: 

# Training FS

In [100]:
max_seq_len=50
rnn_size=2048
input_size = 60
batch_size=64
lr = .0001
train_keep_prob = 1.0
iterations = 10000

In [101]:
tf.reset_default_graph()
# FS
model = Seq2SeqModelFS(max_seq_len, input_size,rnn_size, batch_size, lr,train_keep_prob)
sess = get_session()
sess.run(tf.global_variables_initializer())

rnn_size = 2048
output_size = 60
 state_size = 2048


## Training loop 

In [102]:
score = []
loss = []
max_score = 0
start_time = timeit.default_timer()
for i in range(0,iterations+1):
    encoder_inputs_xyz,decoder_inputs_xyz,seq_len_enc = mini_batch(dsamp_train, seq_len=50, input_size=60, batch_size=64)
    _,train_loss = model.step(sess,encoder_inputs_xyz,decoder_inputs_xyz,seq_len_enc, False)
    if i%100 == 0:
        loss.append(train_loss)
        print("step {0}:  train loss:{1:.4f}".format(i, train_loss),end='')
        end_time = timeit.default_timer()
        print(" using {:.2f} sec".format(end_time-start_time))
        start_time = end_time
        knn_feature = []
        j = 0
        for feature_xyz,labels,seq_len in mini_batch_classify(tr_fea_xyz,tr_label,tr_seq_len,batch_size):
            j +=1
            result = get_feature(model, sess, feature_xyz, len(feature_xyz), seq_len)
            knn_feature.append(result)
        knn_feature = np.vstack(knn_feature)
        test_knn_feature = []
        j = 0
        for feature_xyz,labels,seq_len in mini_batch_classify(te_fea_xyz,te_label,te_seq_len,batch_size):
            j +=1
            result = get_feature(model, sess, feature_xyz,len(feature_xyz),seq_len)
            test_knn_feature.append(result)
        test_knn_feature = np.vstack(test_knn_feature)
        neigh = KNeighborsClassifier(n_neighbors=1,metric='cosine')
        neigh.fit(knn_feature, np.argmax(tr_label,axis=1))
        current_score = neigh.score(test_knn_feature, np.argmax(te_label,axis=1))
        print("KNN test score:",current_score)
        score.append(current_score)
        if current_score>max_score:
            max_score = current_score
            model.saver.save(sess,"ucla_model/FS",global_step=i)
    if i%1000 == 0:
        sess.run(model.learning_rate_decay_op)

step 0:  train loss:121.8500 using 2.18 sec
KNN test score: 0.7408207343412527
step 100:  train loss:12.4162 using 73.64 sec
KNN test score: 0.7796976241900648
step 200:  train loss:10.0450 using 73.88 sec
KNN test score: 0.7948164146868251
step 300:  train loss:9.1190 using 75.43 sec
KNN test score: 0.8056155507559395
step 400:  train loss:9.1730 using 72.85 sec
KNN test score: 0.7991360691144709
step 500:  train loss:9.1789 using 73.34 sec
KNN test score: 0.7883369330453563
step 600:  train loss:8.4541 using 71.92 sec
KNN test score: 0.8099352051835853
step 700:  train loss:8.0992 using 73.01 sec
KNN test score: 0.7991360691144709
step 800:  train loss:7.7977 using 72.07 sec
KNN test score: 0.8099352051835853
step 900:  train loss:7.4648 using 72.73 sec
KNN test score: 0.8185745140388769
step 1000:  train loss:7.7834 using 73.52 sec
KNN test score: 0.8185745140388769
step 1100:  train loss:7.3076 using 76.48 sec
KNN test score: 0.8012958963282938
step 1200:  train loss:7.4040 using 7

KeyboardInterrupt: 

# Get FS model

In [103]:
sess.close()
tf.reset_default_graph()
model = Seq2SeqModelFS(max_seq_len, 60,rnn_size, batch_size, lr,train_keep_prob)
sess = get_session()
model.saver.restore(sess,"ucla_model/FS-900") #"FW-$number of step"

rnn_size = 2048
output_size = 60
 state_size = 2048
INFO:tensorflow:Restoring parameters from ucla_model/FS-900


## Retrieve FS model features

In [104]:
knn_feature = []
j = 0
for feature_xyz,labels,seq_len in mini_batch_classify(tr_fea_xyz,tr_label,tr_seq_len,batch_size):
    j +=1
    result = get_feature(model, sess, feature_xyz, len(feature_xyz), seq_len)
    knn_feature.append(result)
knn_feature = np.vstack(knn_feature)

In [105]:
test_knn_feature = []
j = 0
for feature_xyz,labels,seq_len in mini_batch_classify(te_fea_xyz,te_label,te_seq_len,batch_size):
    j +=1
    result = get_feature(model, sess, feature_xyz,len(feature_xyz),seq_len)
    test_knn_feature.append(result)
test_knn_feature = np.vstack(test_knn_feature)

## Best score from FS

In [106]:
neigh = KNeighborsClassifier(n_neighbors=1,metric='cosine')
neigh.fit(knn_feature,np.argmax(tr_label,axis=1))
t_score = neigh.score(test_knn_feature,np.argmax(te_label,axis=1))
print("after {0} epoch knn score {1:.3f}".format(epoch,t_score))

after 54 epoch knn score 0.819


## AEC for FS

In [107]:
lr = 0.0001
input_size = 2048
batch_size = 16
epochs = 10000

In [108]:
sess.close()
tf.reset_default_graph()
aec_model = AEC(input_size, batch_size, lr)
sess = get_session()
sess.run(tf.global_variables_initializer())

In [109]:
aec_score = 0
for epoch in range(epochs):
    for feature in aec_mini_batch_classify(knn_feature,batch_size):
        train_loss = aec_model.step(sess,feature)
    if epoch % 100 == 0:
        denoise_all = []
        for feature in aec_mini_batch_classify(knn_feature,batch_size):
            denoise = aec_model.step(sess,feature,forward_only=True)
            denoise_all.append(denoise[0])
        see = np.vstack(denoise_all)
            
        test_denoise_all = []
        for feature in aec_mini_batch_classify(test_knn_feature,batch_size):
            denoise = aec_model.step(sess,feature,forward_only=True)
            test_denoise_all.append(denoise[0])
        test_see = np.vstack(test_denoise_all)
        
        neigh = KNeighborsClassifier(n_neighbors=1,metric='cosine')
        neigh.fit(see,np.argmax(tr_label,axis=1))
        t_score = neigh.score(test_see,np.argmax(te_label,axis=1))
        print(t_score)
        if t_score>aec_score:
            aec_score = current_score
            model.saver.save(sess,"ucla_model/FS-AEC")

0.7645788336933045
0.796976241900648
0.7991360691144709
0.8034557235421166
0.8056155507559395
0.8056155507559395
0.796976241900648
0.7991360691144709
0.7991360691144709
0.7926565874730022
0.7991360691144709
0.8034557235421166
0.8012958963282938
0.8034557235421166
0.8056155507559395
0.7991360691144709
0.8012958963282938
0.8120950323974082
0.8056155507559395
0.8034557235421166
0.8120950323974082
0.8099352051835853
0.8034557235421166
0.8120950323974082
0.8120950323974082
0.8099352051835853
0.8185745140388769
0.8120950323974082
0.8099352051835853
0.8056155507559395
0.8012958963282938
0.8056155507559395
0.8077753779697624
0.8142548596112311
0.8077753779697624
0.8056155507559395
0.8120950323974082
0.8142548596112311
0.8120950323974082
0.816414686825054
0.8056155507559395
0.8099352051835853
0.816414686825054
0.8120950323974082
0.8142548596112311
0.816414686825054
0.8142548596112311
0.8142548596112311
0.8142548596112311
0.8077753779697624
0.8207343412526998
0.8056155507559395


KeyboardInterrupt: 